In [1]:
CNY_USD = 0.1389
RUB_USD = 0.010309087
CAD_USD = 0.74

Today = '31.12.2023'

# Group = 'EuroChem'
Group = 'SUEK'


Data_Debt = 'WHWEEK_quantumDebt_2023_12' + '.xlsx'
Data_Forwards = 'WHWEEK_quantumForwards_2023_12' + '.xlsx'
Data_Swaps = 'WHWEEK_quantumSwaps_2023_12' + '.xlsx'
Sheet_in_data_file = 'Export Worksheet'

Mapping_file = 'C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP\\Методология\\Mapping.xlsx'
Sheet_in_mapping_file = 'mapping'

from datetime import date

Output_Debt = "_".join([str(date.today()), Group, 'quantum', 'Debt.xlsx'])
Output_Deriv = "_".join([str(date.today()), Group, 'quantum', 'Deriv.xlsx'])
Sheet_in_output_Deriv = 'Deriv'
Sheet_in_output_Debt = 'Debt'

In [15]:
import pandas as pd
import numpy as np

data_Debt_export = pd.read_excel(Data_Debt, sheet_name = Sheet_in_data_file)
data_Debt_export[['entity', 'counterparty']] = data_Debt_export[['entity', 'counterparty']].apply(lambda x: x.str.lower())

data_Forwards_export = pd.read_excel(Data_Forwards, sheet_name = Sheet_in_data_file)
data_Forwards_export.entity = data_Forwards_export.entity.str.lower()

data_Swaps_export = pd.read_excel(Data_Swaps, sheet_name = Sheet_in_data_file)
data_Swaps_export.entity = data_Swaps_export.entity.str.lower()

c:\Users\KlimovaAnnaA\AppData\Local\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\KlimovaAnnaA\AppData\Local\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [34]:
data_mapping = pd.read_excel(Mapping_file, sheet_name = Sheet_in_mapping_file)
data_mapping[['Unnamed: 23', 'Code', 'Cpty_name', 'Code2']] = data_mapping[['Unnamed: 23', 'Code', 'Cpty_name', 'Code2']].apply(lambda x: x.str.lower())

Фильтрация данных

In [35]:
min_data = data_Debt_export.effectiveFrom.min() # Присваивание переменной значения мин даты в столбце effectiveFrom
data_Debt_work = data_Debt_export.query('holding == @Group & effectiveFrom == @min_data').reset_index(drop=True) # Фильтрация данных по столбцу holding (переменная Group) и effectiveFrom

merge_df = data_Debt_work.merge(data_mapping, how = 'left', left_on = 'entity', right_on = 'Unnamed: 23')[['Unnamed: 24', 'Unnamed: 25']]
data_Debt_work = pd.concat([data_Debt_work, merge_df], axis=1).rename(columns={'Unnamed: 24': 'Entity', 'Unnamed: 25': 'Entity_group'})
# Merge столбца entity по XYZ в mapping

In [39]:
# data_Debt_work[['entity', 'Entity', 'Entity_group']]

,entity,Entity,Entity_group
0,xxtsu,АО СУЭК,SUEK RU
1,xxtsu,АО СУЭК,SUEK RU
2,xxtsu,АО СУЭК,SUEK RU
3,xxtsu,АО СУЭК,SUEK RU
4,xxtsu,АО СУЭК,SUEK RU
...,...,...,...
4942,xxtsu18,АО Кузбассэнерго,SGC
4943,xxtsu18,АО Кузбассэнерго,SGC
4944,xxtsu18,АО Кузбассэнерго,SGC
4945,xxtsu18,АО Кузбассэнерго,SGC


Выполнение цикла для рассчета amount, amountUSD и присваивания значения counterparty для каждого DealSet

In [40]:
deals_data = data_Debt_work[['Entity', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet']] # Выбор нужного разреза данных
deals_data_Bonds = deals_data[deals_data.dealSet.str.contains('Bond')] # Фильтрация разреза по значениям в DealSet, которые содержат слово Bond

Bonds_list = deals_data_Bonds.dealSet.unique()

for Bond in Bonds_list:
    try:
        Bond_data = deals_data_Bonds.query('dealSet ==@Bond') # Отбор данных одного Bond

        for column in Bond_data[['amount', 'amountUSD']]:

            Traded_data_sum = Bond_data.query('instrumentOwner =="EC_TRADED_BONDS"')[column].sum()
            Issued_data = Bond_data.query('instrumentOwner =="EC_ISSUED_BONDS"')[column]

            # Внесение нового значения amount в issued bonds
            Bond_data.loc[Bond_data['instrumentOwner'] =="EC_ISSUED_BONDS", column] = Traded_data_sum + Issued_data 

        # Внесение нового значения counterparty в traded bonds
        Issied_data_entity = Bond_data.query('instrumentOwner =="EC_ISSUED_BONDS"').Entity.squeeze()
        Bond_data.loc[Bond_data['instrumentOwner'] =="EC_TRADED_BONDS", 'counterparty'] = Issied_data_entity

        deals_data_Bonds.loc[deals_data_Bonds['dealSet'] == Bond] = Bond_data
    except:
        continue

deals_data.loc[deals_data.dealSet.str.contains('Bond')] = deals_data_Bonds
data_Debt_work[['Entity', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet']] = deals_data
# Запись выполненного цикла в в основные данные

In [41]:
day = pd.to_datetime(Today, dayfirst = True)
data_Debt_work['Days'] = 0

data_Debt_work['Days'] = data_Debt_work.loc[data_Debt_work['actionDate'] != np.NaN, 'actionDate'] - day
data_Debt_work['Days'] = data_Debt_work['Days'].dt.days
data_Debt_work['Period'] = '2Y+'

data_Debt_work.loc[pd.to_numeric(data_Debt_work['Days']) < 730, 'Period'] = '1Y-2Y'
data_Debt_work.loc[pd.to_numeric(data_Debt_work['Days']) < 365, 'Period'] = '6M-1Y'
data_Debt_work.loc[pd.to_numeric(data_Debt_work['Days']) < 182, 'Period'] = '3M-6M'
data_Debt_work.loc[pd.to_numeric(data_Debt_work['Days']) < 91, 'Period'] = '1M-3M'
data_Debt_work.loc[pd.to_numeric(data_Debt_work['Days']) < 30, 'Period'] = '<1M'

data_Debt_work.counterparty = data_Debt_work.counterparty.str[:-5]
data_Debt_work['CompCode'] = data_Debt_work\
    .merge(data_mapping, how = 'left', left_on = 'counterparty', right_on = 'Cpty_name')['Code2']\
    .replace({np.NaN : 'External'})
data_Debt_work['Counterparty_Group'] = data_Debt_work\
    .merge(data_mapping, how = 'left', left_on = 'CompCode', right_on = 'Code')['Business_segment']\
    .replace({np.NaN : 'External'})
data_Debt_work.loc[(data_Debt_work['cpartyType'] == 'Internal') \
                 & (data_Debt_work['Counterparty_Group'] == 'External'),\
                                                                'Counterparty_Group'] = 'Internal'
# Merge counterparty с mapping 

In [42]:
# manual_map = data_Debt_export.loc[data_Debt_export['cpartyType'] == 'Internal', 'counterparty'].drop_duplicates()
# # manual_map.to_excel('Manual_map.xlsx', index=False)
# manual_map

# data_Debt_export.loc[data_Debt_export['cpartyType'] == 'Internal', 'counterpartyAlt']

In [43]:
# data_Debt_export.to_excel(Output_Debt, sheet_name = Sheet_in_output_Debt, index = False)
# Полные данные для проверки качества выполнения кода

Запись данных в файл Excel

In [44]:
data_Debt_work['Source'] = 'Quantum'
data_Debt_work[['entity', 'counterparty', 'CompCode']] = data_Debt_work[['entity', 'counterparty', 'CompCode']].apply(lambda x: x.str.upper())

data_Debt_work_for_print = data_Debt_work[['Entity', 'Entity_group','counterparty',\
                                               'Counterparty_Group','amount','eventCurrency','amountUSD','actionDate','Days',\
                                                'Period','dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]
# Отбор нужных столбцов

data_Debt_work_for_print = data_Debt_work_for_print.rename(columns={'amount': 'amoutn outstanding',\
                                                                        'eventCurrency': 'Currency',\
                                                                        'amountUSD': 'amount USD eq',\
                                                                        'actionDate': 'termEnd'})
# Переименование столбцов

data_Debt_work_for_print.to_excel(Output_Debt, sheet_name = Sheet_in_output_Debt, index = False)
# Запись в файл

Работа с Forwards и Swaps:

In [45]:
data_Forwards_export['holdingEntity'] = data_Forwards_export['holdingEntity'].replace({'XXTSU': 'SUEK'})
data_Forwards_export = data_Forwards_export[data_Forwards_export['holdingEntity'] == Group].reset_index(drop=True)
data_Swaps_export['holdingEntity'] = data_Swaps_export['holdingEntity'].replace({'XXTSU': 'SUEK'})
data_Swaps_export = data_Swaps_export[data_Swaps_export['holdingEntity'] == Group].reset_index(drop=True)
# Знамена значений в стодбце holdingEntity и фильтр по Group

Проверка вхождений dealNo в Debt и остановка процесса в случае отсутвия данных:

In [46]:
Swap_dealNo_for_del_list = list(set(data_Swaps_export['dealNo']).intersection(set(data_Debt_export['dealNo'])))
Forward_dealNo_for_del_list = list(set(data_Forwards_export['dealNo']).intersection(set(data_Debt_export['dealNo'])))

data_Swaps_export = data_Swaps_export[~(data_Swaps_export.dealNo.isin(Swap_dealNo_for_del_list))]
data_Forwards_export = data_Forwards_export[~(data_Forwards_export.dealNo.isin(Forward_dealNo_for_del_list))]

assert ((len(data_Swaps_export) == 0) and (len(data_Forwards_export) == 0)) == False

In [47]:
Data_for_loop = data_Forwards_export[['entity','holdingEntity','counterparty','dealNo', 'payFaceValue', 'payFXCurrency', 'recFaceValue', 'recFXCurrency', 'maturityDate']]

for i in range(0, len(Data_for_loop)):
    row_list = list(Data_for_loop.loc[i])
    Data_for_loop.loc[len(Data_for_loop)] = row_list[:4] + row_list[6:8] + row_list[6:]
    Data_for_loop.loc[i,'payFaceValue'] = Data_for_loop.loc[i,'payFaceValue'] * -1
Data_for_loop = Data_for_loop.sort_values('dealNo')
# цикл создания новых строк

merge_df = Data_for_loop.merge(data_mapping, how = 'left', left_on = 'entity', right_on = 'Unnamed: 23')[['Unnamed: 24', 'Unnamed: 25']]
Data_for_loop = pd.concat([Data_for_loop, merge_df], axis=1).rename(columns={'Unnamed: 24': 'Entity', 'Unnamed: 25': 'Entity_group'})
# Merge столбца entity по XYZ в mapping

day = pd.to_datetime(Today, dayfirst = True)
Data_for_loop['Days'] = 0

Data_for_loop['Days'] = Data_for_loop.loc[Data_for_loop['maturityDate'] != np.NaN, 'maturityDate'] - day
Data_for_loop['Days'] = Data_for_loop['Days'].dt.days
Data_for_loop['Period'] = '2Y+'

Data_for_loop.loc[pd.to_numeric(Data_for_loop['Days']) < 730, 'Period'] = '1Y-2Y'
Data_for_loop.loc[pd.to_numeric(Data_for_loop['Days']) < 365, 'Period'] = '6M-1Y'
Data_for_loop.loc[pd.to_numeric(Data_for_loop['Days']) < 182, 'Period'] = '3M-6M'
Data_for_loop.loc[pd.to_numeric(Data_for_loop['Days']) < 91, 'Period'] = '1M-3M'
Data_for_loop.loc[pd.to_numeric(Data_for_loop['Days']) < 30, 'Period'] = '<1M'
# Создание Days и Period

Data_for_loop['USD_equiv'] = 0
for i in range (0,len(Data_for_loop)):
    if Data_for_loop.loc[i, 'payFXCurrency'] == 'CNY':
        Data_for_loop.loc[i, 'USD_equiv'] = Data_for_loop.loc[i, 'payFaceValue'] * CNY_USD
    elif Data_for_loop.loc[i, 'payFXCurrency'] == 'RUB':
        Data_for_loop.loc[i, 'USD_equiv'] = Data_for_loop.loc[i, 'payFaceValue'] * RUB_USD
    elif Data_for_loop.loc[i, 'payFXCurrency'] == 'USD':
        Data_for_loop.loc[i, 'USD_equiv'] = Data_for_loop.loc[i, 'payFaceValue']
    elif Data_for_loop.loc[i, 'payFXCurrency'] == 'CAD':
        Data_for_loop.loc[i, 'USD_equiv'] = Data_for_loop.loc[i, 'payFaceValue'] * CAD_USD
# создание USD_equiv

new_columns = ['Company', 'Settlement', 'Notional Amount (USD)', 'Unnamed', 'source']
Data_for_loop = Data_for_loop.reindex(columns=(Data_for_loop.columns.tolist() + new_columns))
Data_for_loop['source'] = 'Quantum'
# Создание пустых столбцов и столбца ресурса

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_2820\2331839492.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data_for_loop.loc[len(Data_for_loop)] = row_list[:4] + row_list[6:8] + row_list[6:]
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_2820\2331839492.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data_for_loop.loc[len(Data_for_loop)] = row_list[:4] + row_list[6:8] + row_list[6:]
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_2820\2331839492.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '21676144.053375' 

In [48]:
Data_for_loop_Swap = data_Swaps_export[['entity','holdingEntity','counterparty','dealNo', 'payFaceValue', 'payFXCurrency', 'recFaceValue', 'recFXCurrency', 'maturityDate']]

for i in range(0, len(Data_for_loop_Swap)):
    row_list = list(Data_for_loop_Swap.loc[i])
    Data_for_loop_Swap.loc[len(Data_for_loop_Swap)] = row_list[:4] + row_list[6:8] + row_list[6:]
    Data_for_loop_Swap.loc[i,'payFaceValue'] = Data_for_loop_Swap.loc[i,'payFaceValue'] * -1
Data_for_loop_Swap = Data_for_loop_Swap.sort_values('dealNo')
# цикл создания новых строк

merge_df = Data_for_loop_Swap.merge(data_mapping, how = 'left', left_on = 'entity', right_on = 'Unnamed: 23')[['Unnamed: 24', 'Unnamed: 25']]
Data_for_loop_Swap = pd.concat([Data_for_loop_Swap, merge_df], axis=1).rename(columns={'Unnamed: 24': 'Entity', 'Unnamed: 25': 'Entity_group'})
# Merge столбца entity по XYZ в mapping

day = pd.to_datetime(Today, dayfirst = True)
Data_for_loop_Swap['Days'] = 0

Data_for_loop_Swap['Days'] = Data_for_loop_Swap.loc[Data_for_loop_Swap['maturityDate'] != np.NaN, 'maturityDate'] - day
Data_for_loop_Swap['Days'] = Data_for_loop_Swap['Days'].dt.days
Data_for_loop_Swap['Period'] = '2Y+'

Data_for_loop_Swap.loc[pd.to_numeric(Data_for_loop_Swap['Days']) < 730, 'Period'] = '1Y-2Y'
Data_for_loop_Swap.loc[pd.to_numeric(Data_for_loop_Swap['Days']) < 365, 'Period'] = '6M-1Y'
Data_for_loop_Swap.loc[pd.to_numeric(Data_for_loop_Swap['Days']) < 182, 'Period'] = '3M-6M'
Data_for_loop_Swap.loc[pd.to_numeric(Data_for_loop_Swap['Days']) < 91, 'Period'] = '1M-3M'
Data_for_loop_Swap.loc[pd.to_numeric(Data_for_loop_Swap['Days']) < 30, 'Period'] = '<1M'
# Создание Days и Period

Data_for_loop_Swap['USD_equiv'] = 0
for i in range (0,len(Data_for_loop_Swap)):
    if Data_for_loop_Swap.loc[i, 'payFXCurrency'] == 'CNY':
        Data_for_loop_Swap.loc[i, 'USD_equiv'] = Data_for_loop_Swap.loc[i, 'payFaceValue'] * CNY_USD
    elif Data_for_loop_Swap.loc[i, 'payFXCurrency'] == 'RUB':
        Data_for_loop_Swap.loc[i, 'USD_equiv'] = Data_for_loop_Swap.loc[i, 'payFaceValue'] * RUB_USD
    elif Data_for_loop_Swap.loc[i, 'payFXCurrency'] == 'USD':
        Data_for_loop_Swap.loc[i, 'USD_equiv'] = Data_for_loop_Swap.loc[i, 'payFaceValue']
# создание USD_equiv

new_columns = ['Company', 'Settlement', 'Notional Amount (USD)', 'Unnamed', 'source']
Data_for_loop_Swap = Data_for_loop_Swap.reindex(columns=(Data_for_loop_Swap.columns.tolist() + new_columns))
Data_for_loop_Swap['source'] = 'Quantum'
# Создание пустых столбцов и столбца ресурса

In [49]:
data = Data_for_loop[['Entity','Entity_group','holdingEntity','Company',\
                    'counterparty','dealNo','payFXCurrency','payFaceValue',\
                    'USD_equiv','Settlement','Notional Amount (USD)',\
                    'Unnamed','maturityDate','Days','Period','source']]
data.Entity = data.Entity.str.upper()

data_Swap = Data_for_loop_Swap[['Entity','Entity_group','holdingEntity','Company',\
                    'counterparty','dealNo','payFXCurrency','payFaceValue',\
                    'USD_equiv','Settlement','Notional Amount (USD)',\
                    'Unnamed','maturityDate','Days','Period','source']]
data_Swap.Entity = data_Swap.Entity.str.upper()
# Выбор столбцов

data_to_excel = pd.concat([data, data_Swap], axis=0)
# Объединение таблиц вниз
data_to_excel.to_excel(Output_Deriv, sheet_name = Sheet_in_output_Deriv, index = False)
# Запись в файл 

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_2820\340167860.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Entity = data.Entity.str.upper()
